# agglomerative_CNN_clustering

(Hierarchical clustering)
- Affinity propagation appears to do quite a good job at grouping together individual clusters (and helps intuit the ballpark number of catagories), however, the clustering of galaxies is most likely hierarchical in structure (clustering-wise at least). 
- Using agglomerative clustering to construct clusters

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns

from sklearn.cluster import AgglomerativeClustering
import cluster_evaluate

### Loading set of features 
- Also weights in case we want to manipulate feature space to rank pca components that best describe the variance in the data
- Several feature representations were generated, so selecting combination of output features from CNN and the number of features these were compressed to by the pca.

In [ ]:
# define the model
agglo = AgglomerativeClustering(n_clusters=10, linkage='ward')
# fit the model
agglo_labels = agglo.fit_predict(compressed_features)

In [ ]:
for cluster in np.unique(agglo_labels):
    print('Cluster : {}'.format(cluster))
    cluster_evaluate.plot_cluster(x[agglo_labels == cluster][0:10])
    plt.show()